<a href="https://colab.research.google.com/github/Dzechko/DL_from_scratch/blob/main/00_Foundation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Difference between python list operation and numpy array operation

In [1]:
ls1 = [1,2,3]
ls2 = [3,4,5]
print(ls1+ls2)
#print(ls1*ls2)
#cant multiply two lists

[1, 2, 3, 3, 4, 5]


In [2]:
import numpy as np
np1 = np.array([3,4,5])
np2 = np.array([4,5,6])
print(np1+np2)
print(np1*np2)


[ 7  9 11]
[12 20 30]


In [3]:
a= np.array([[1,2],[3,4]])
print(a)
print(a.sum(axis=0))
print(a.sum(axis=1))

[[1 2]
 [3 4]]
[4 6]
[3 7]


In [4]:
a= np.array([[1,2],[3,4]])
b= np.array([7,8])
print(a+b)

[[ 8 10]
 [10 12]]


AN EXAMPLE OF CALLABLE

In [5]:
from typing import Callable

def apply_func(func: Callable[[int],int], x:int)->int:
  return func(x)

def square(x:int)->int:
  return x*x

print(apply_func(square,5))


25


DERIVATIVE FUNCTION USING CALLABLE

In [6]:
import numpy as np
from typing import Callable

ndarray = np.array

def deriv(func: Callable[[ndarray],ndarray],
          input_:ndarray,
          delta: float=0.001)->ndarray:

          return (func(input_+delta)-func(input_-delta))/(2*delta)



NESTED FUNCTIONS

In [7]:
from typing import List, Callable
import numpy as np

array_function = Callable[[np.ndarray], np.ndarray]
Chain = List[array_function]

def chain_length_2(chain: Chain, a: np.ndarray) -> np.ndarray:
    assert len(chain) == 2, "Chain length must be 2"
    f1 = chain[0]
    f2 = chain[1]
    return f1(f2(a))


In [8]:
import numpy as np
ndarray = np.array
def sigmoid(x:ndarray)->ndarray:
  return 1/(1+ np.exp(-x))


In [9]:
def multiple_inputs_add(x:ndarray,
                        y:ndarray,
                        sigma:array_function)->ndarray:

                        assert x.shape==y.shape

                        a = x + y
                        return sigma(a)



In [10]:
a =  np.arange(1,5,1)
b = np.arange(2,10,2)
print(a)
print(b)

[1 2 3 4]
[2 4 6 8]


In [11]:
multiple_inputs_add(a,b,square)

array([  9,  36,  81, 144])

In [12]:
def multiple_input_add_backward(x:ndarray,
                                y:ndarray,
                                sigma:array_function)->float:

                              a = x+y
                              dsda = deriv(sigma,a)
                              dadx = 1
                              dadx = 1
                              return dsda*dadx,dsda*dadx




In [13]:
p = np.array([2,4,7])
q = np.array([5,7,1])
multiple_input_add_backward(p,q,square)

(array([14., 22., 16.]), array([14., 22., 16.]))

In [14]:
def multiple_input_mul_backward(x:ndarray,
                                y:ndarray,
                                sigma:array_function)->float:

                                a = x*y
                                dsda = deriv(sigma,a)
                                dadx = y
                                dady = x
                                return dsda*dadx,dsda*dady


In [15]:
multiple_input_mul_backward(p,q,square)

(array([100., 392.,  14.]), array([ 40., 224.,  98.]))

functions with multiple vector inputs

In [16]:
d = ndarray([[1,2,3],
             [3,4,5]])
e = ndarray([[8,9,1],
             [2,7,5]])
#no of rows
d.shape[0]
#no of colums
d.shape[1]
d.shape

(2, 3)

In [17]:
def matmul_forward(X:ndarray,
                   W:ndarray)->ndarray:

                   assert X.shape[1] == W.shape[0]

                   N = np.dot(X,W)

                   return N


In [18]:
G = np.array([[1,2,3,4]])
E = np.array([[1],
             [2],
             [3],
             [4]])
E.shape,G.shape

((4, 1), (1, 4))

In [19]:
matmul_forward(G,E)

array([[30]])

In [20]:
def matmul_backward_first(X:ndarray,
                          W:ndarray)->ndarray:

          dndx = np.transpose(W,(1,0))
          dndw = np.transpose(X,(1,0))

          return dndx,dndw


In [21]:
matmul_backward_first(G,E)

(array([[1, 2, 3, 4]]),
 array([[1],
        [2],
        [3],
        [4]]))

In [22]:
def  matrix_forward_extra(X:ndarray,
                          W:ndarray,
                          sigma:array_function)->ndarray:

                          assert X.shape[1] == W.shape[0]

                          # matrix multiplication

                          N = np.dot(X, W)
                          # function

                          S = sigma(N)

                          return S

In [23]:
matrix_forward_extra(G,E,square)

array([[900]])

In [24]:
def matrix_function_backward_1(X:ndarray,
                               W:ndarray,
                               sigma:array_function)->ndarray:

                               # forward
                               assert X.shape[1]==W.shape[0]
                               N = np.dot(X, W)
                               S = sigma(N)

                               #backward
                               dsdN = deriv(sigma,N)
                               dndX = np.transpose(W,(1,0))
                               dndW = np.transpose(X,(1,0))

                               return np.dot(dsdN, dndX)

In [25]:
matrix_function_backward_1(G,E,square)

array([[ 60., 120., 180., 240.]])

In [26]:
def matrix_function_forward_sum(X:ndarray,
                                W:ndarray,
                                sigma:array_function)->float:

                                assert X.shape[1]==W.shape[0]

                                N = np.dot(X, W)
                                S = sigma(N)
                                L = np.sum(S)

                                return L

In [27]:
matrix_function_forward_sum(G,E,square)

900

In [28]:
def matrix_function_backward_sum_1(X:ndarray,
                                   W:ndarray,
                                   sigma:array_function)->ndarray:

                                   assert X.shape[1]==W.shape[0]
                                   N = np.dot(X, W)
                                   S = sigma(N)
                                   L = np.sum(S)

                                   #backward
                                   dldS = np.ones_like(S)
                                   dsdN = deriv(sigma,N)
                                   dldN = dldS*dsdN
                                   dndX = np.transpose(W,(1,0))
                                   dLdX = np.dot(dsdN, dndX)


                                   return dLdX




In [29]:
matrix_function_backward_sum_1(G,E,square) #both are same dot product and elementary wise operation (dlds is ones)

array([[ 60., 120., 180., 240.]])

In [30]:
np.random.seed(190204)
X = np.random.randn(3,3)
Y = np.random.randn(3,2)
print(X)
print(Y)


[[-1.57752816 -0.6664228   0.63910406]
 [-0.56152218  0.73729959 -1.42307821]
 [-1.44348429 -0.39128029  0.1539322 ]]
[[ 0.75510818  0.25562492]
 [-0.56109271 -0.97504841]
 [ 0.98098478 -0.95870776]]


In [31]:
print("L:")
print(matrix_function_forward_sum(X,Y,sigmoid))

L:
2.3754951014116954


In [32]:
print("dldx")
print(matrix_function_backward_sum_1(X,Y,sigmoid))

dldx
[[ 0.2488887  -0.37478057  0.01121962]
 [ 0.12604152 -0.27807404 -0.13945837]
 [ 0.22992798 -0.36623443 -0.02252592]]
